#### Testing parallelized pairs indices generation with pycuda

Pairs generation for all events following combinations

In [31]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [32]:
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)
cumul_arr = numpy.zeros(base_len+1)
cumul_arr[0] = 0
cumul_arr[1:] = numpy.cumsum(base_arr)
start = cumul_arr[:-2]
stop = cumul_arr[1:-1]
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*(lengths+1)/2)
lengths_arr = numpy.array([base_len-1]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)
start = start.astype(numpy.int32)

In [33]:
pairs_lengths.shape

(32,)

In [34]:
mod = SourceModule('''
__global__ void comb_indices(int* left,int* right,int* start,int* length,int* lengths,int* pairs_lengths)
{
    int i = threadIdx.x + blockDim.x*blockIdx.x;
    int j = threadIdx.y + blockDim.y*blockIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if ( i<length[0] && j< lengths[i] && k<=j) 
    {
        left[pairs_lengths[i]+lengths[i]*(lengths[i]+1)/2-(j*(j+1)/2)-k-1] = lengths[i]-1-j + start[i];
        right[pairs_lengths[i]+lengths[i]*(lengths[i]+1)/2 - (j*(j+1)/2) -k -1] = lengths[i]-k + start[i];
    }
}
''')
func = mod.get_function("comb_indices")

C:\Users\Jaydeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  del sys.path[0]


In [35]:
left = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)

In [36]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(lengths_arr),
    cuda.In(lengths), cuda.In(pairs_lengths), block=(1, 16, 16), grid=(base_len-1, 1))

In [37]:
print(right)

[ 1  2  3  4  3  4  4  5  6  6  7  8  8  9 10 11 10 11 11 12 13 14 15 16 13
 14 15 16 14 15 16 15 16 16 17 18 19 20 21 18 19 20 21 19 20 21 20 21 21 22
 23 24 25 26 23 24 25 26 24 25 26 25 26 26 27 28 28 29 30 31 32 33 32 33 33
 34 35 36 37 38 35 36 37 38 36 37 38 37 38 38 39 40 41 40 41 41 42 43 44 45
 43 44 45 44 45 45 46 47 48 49 50 47 48 49 50 48 49 50 49 50 50 51 52 53 52
 53 53 54 55 56 55 56 56 57 58 59 59 60 61 62 61 62 62 63 64 65 66 67 68 69
 66 67 68 69 67 68 69 68 69 69 70 71 72 73 71 72 73 72 73 73 74 75 76 75 76
 76 77 78 79 80 81 78 79 80 81 79 80 81 80 81 81 82 83 83 84]


In [38]:
for i in range(6):
    print("Event: {} \n Left:{} \n Right:{} \n".format(i, left[pairs_lengths[i]:pairs_lengths[i+1]], right[pairs_lengths[i]:pairs_lengths[i+1]]))

Event: 0 
 Left:[] 
 Right:[] 

Event: 1 
 Left:[0] 
 Right:[1] 

Event: 2 
 Left:[] 
 Right:[] 

Event: 3 
 Left:[1 1 1 2 2 3] 
 Right:[2 3 4 3 4 4] 

Event: 4 
 Left:[4 4 5] 
 Right:[5 6 6] 

Event: 5 
 Left:[6 6 7] 
 Right:[7 8 8] 



In [39]:
print(left)

[ 0  1  1  1  2  2  3  4  4  5  6  6  7  8  8  8  9  9 10 11 11 11 11 11 12
 12 12 12 13 13 13 14 14 15 16 16 16 16 16 17 17 17 17 18 18 18 19 19 20 21
 21 21 21 21 22 22 22 22 23 23 23 24 24 25 26 26 27 28 29 30 30 30 31 31 32
 33 33 33 33 33 34 34 34 34 35 35 35 36 36 37 38 38 38 39 39 40 41 41 41 41
 42 42 42 43 43 44 45 45 45 45 45 46 46 46 46 47 47 47 48 48 49 50 50 50 51
 51 52 53 53 53 54 54 55 56 57 57 58 59 59 59 60 60 61 62 63 64 64 64 64 64
 65 65 65 65 66 66 66 67 67 68 69 69 69 69 70 70 70 71 71 72 73 73 73 74 74
 75 76 76 76 76 76 77 77 77 77 78 78 78 79 79 80 81 81 82 83]
